In [ ]:
from PIL import Image, ImageEnhance
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd

import requests
import json
import re
import csv

directory_path = os.getcwd()

csv_path = os.path.join(directory_path, 'pix2pix_models\\site_area.csv')
masked_image_path = os.path.join(directory_path, 'pix2pix_models\\data\\mapbox_clear_data\\input\\')
generated_image_path = os.path.join(directory_path, 'pix2pix_models\\outputs\\clear_output\\') 

# Read the CSV file
data = pd.read_csv(csv_path)

# Function to extract the numeric part of the filename
def extract_numeric_part(filename):
    numeric_part = ''.join(filter(str.isdigit, filename))
    return int(numeric_part) if numeric_part else None

def create_binary_mask(arr, target_color, threshold=30):
    lower_bound = np.array(target_color) - threshold
    upper_bound = np.array(target_color) + threshold
    mask = (arr[:, :, :3] >= lower_bound) & (arr[:, :, :3] <= upper_bound)
    return np.all(mask, axis=-1)

def extract_building_regions(arr, target_color, threshold=10):
    lower_bound = np.array(target_color) - threshold
    upper_bound = np.array(target_color) + threshold
    mask = (arr[:, :, :3] >= lower_bound) & (arr[:, :, :3] <= upper_bound)
    return np.all(mask, axis=-1)

def find_max_building_storeys(gpr):
    max_building_storeys= 0
    if gpr >= 0 and gpr < 1.4:
        max_building_storeys = 5
    elif gpr >= 1.4 and gpr < 1.6:
        max_building_storeys = 12
    elif gpr >= 1.6 and gpr < 2.1:
        max_building_storeys = 24
    elif gpr >= 2.1 and gpr < 2.8:
        max_building_storeys = 36
    elif gpr >= 2.8:
        max_building_storeys = 48 ## by right got no limit
    return max_building_storeys


ok_ratio = []
ok_images = []
all_images = []
all_ratio =[]

# Iterate through the images in the generated_image_path
for image_file in os.listdir(generated_image_path):
    if image_file.endswith('.png'):
        image_index = extract_numeric_part(image_file)

        # Construct the path for the corresponding masked image
        mask_image_filename = f"{image_index}.png"
        mask_image = os.path.join(masked_image_path, mask_image_filename)
        generated_image = os.path.join(generated_image_path, image_file)

        # Check if the image index matches any index in the CSV
        matched_row = data[data['Index'] == image_index]
        if not matched_row.empty:
            # Extract the GPR value for the matched row
            gpr_value = matched_row['GPR'].iloc[0]
            actual_site_area = matched_row['Area'].iloc[0]
            actual_site_area = actual_site_area.replace(',', '')
            actual_site_area = float(actual_site_area[:-4])
            gpr_value = float(gpr_value)
            

            print(f'Image: {image_file}, GPR: {gpr_value}')
            mask_image = Image.open(mask_image)  # 512x512
            generated_image =  Image.open(generated_image)
            mask_array = np.array(mask_image)
            generated_array = np.array(generated_image)
            mask_color = [255, 255, 0]  # bright yellow
            site_mask = create_binary_mask(mask_array, mask_color)
            site_area_array = generated_array.copy()
            site_area_array[~site_mask] = [255, 255, 255, 255] # making non-masked region white RMB ITS 4 CHANNELS NOW
            site_area_image = Image.fromarray(site_area_array)
            
            plt.figure(figsize=(20, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(generated_image)
            plt.title('Original Image')
            plt.axis('off')
            plt.subplot(1, 3, 2)
            plt.imshow(site_area_image)
            plt.title('Site Image')
            plt.axis('off')
            plt.show()

            mask_color = [250, 8, 162]  # bright pink
            building_mask = extract_building_regions(site_area_array, mask_color)
            buildings_image = Image.fromarray(building_mask)


            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(generated_image)
            plt.title('Original Image')
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(buildings_image, cmap='gray')
            plt.title('Buildings Image')
            plt.axis('off')
            plt.show()

            # find site:building area ratio
            mask_pixels = np.sum(site_mask)
            ratio = float(mask_pixels/actual_site_area)
            all_ratio.append(ratio)
            all_images.append(image_file)
            if ratio <= 0.8 and ratio >= 0.68:
                ok_ratio.append(ratio)
                ok_images.append(image_file)
print('All_Ratio', all_ratio)
print('All_Images', ok_images)
print('OK_Ratio', ok_ratio)
print('OK_Images', ok_images)
\
